In [1]:
import os
import re
import tempfile
from monty.tempfile import ScratchDir

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_google_genai import GoogleGenerativeAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.atomate2.agents import MLFFMDAgent
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4-turbo"

In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0.0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool(
    description=re.sub(
        r"\s+",
        " ",
        """A Python shell. Use this to execute python commands.
        Input should be a valid python command.
        
        TIPS:
        - If you want to see the output of a value, you should print it out
        with `print(...)`.
        - If you write any file to a local file system, you should ALWAYS 
        print the paths to the files as well.
        - If you plot any graph, you should show the graph, save it to local 
        and print the path to the image file.
        - Combine all your output string together and print once together in
        JSON format at the end of the code.
        """
    )
)]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MLFFMDAgent(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
    r"\s+",
    " ",
    """You name is LLaMP and you are a helpful agent that can consult 
    materials-related data through Materials Project (MP) database, arXiv, 
    Wikipedia, and run calculations/simulations via a python REPL and MP atomate2 
    computational workflow.     
        
    If you get an error, debug your code and try again. Only use 
    the output of your code to answer the question. Ask user to clarify their 
    queries if needed. Please note that you don't have direct control over MP but 
    through multiple assistant agents to help you. You need to provide complete 
    context in the input for assistants to do their job. REFINE and CRITQUE the 
    output of the assistants if needed. You can end the conversation by saying "Bye".
    """,
    )
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:



pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

query = """
Give me the formation energies of the following materials in their most stable bulk form: Ag2O3, MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3. If there are multiple values, please give me the smallest value.
"""

# with ScratchDir(rootpath=os.getcwd()):
# with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
#     os.chdir(tmpdir)
#     while True:
query = query or input("Query:")
print(f"Query: {query}")
response = agent_executor.invoke(
    {
        "input": query,
    }
)
# if re.search(pattern, response["output"]):
#     break
# query = None

Query: 
Give me the formation energies of the following materials in their most stable bulk form: Ag2O3, MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3. If there are multiple values, please give me the smallest value.



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPThermoExpert
Action Input: What are the formation energies of Ag2O3, MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3 in their most stable bulk form?Thought: Do I need to use a tool? Yes
Action: MPThermoExpert
Action Input: What are the formation energies of Ag2O3, MgZn2, GaAs, GaN, SiC, BN, InSe, MoS2, BaTiO3, CsPbI3 in their most stable bulk form?

> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "search_materials_thermo__get",
  "action_input": {
    "formula": "Ag2O3,MgZn2,GaAs,GaN,SiC,BN,InSe,MoS2,BaTiO3,CsPbI3",
    "fields": "material_id,formula_pretty,formation_energy_per_atom,is_stable",
    "is_stable": true
  }
}
```Action:
```json
{
  "action

Retrieving ThermoDoc documents:   0%|          | 0/27 [00:00<?, ?it/s]

[{'formula_pretty': 'InSe', 'material_id': 'mp-20485', 'formation_energy_per_atom': -0.676726512031248, 'is_stable': True}, {'formula_pretty': 'InSe', 'material_id': 'mp-20485', 'formation_energy_per_atom': -0.676726512031248, 'is_stable': True}, {'formula_pretty': 'InSe', 'material_id': 'mp-22691', 'formation_energy_per_atom': -0.7462709086718746, 'is_stable': True}, {'formula_pretty': 'MoS2', 'material_id': 'mp-2815', 'formation_energy_per_atom': -1.2015554987500003, 'is_stable': True}, {'formula_pretty': 'MoS2', 'material_id': 'mp-1434', 'formation_energy_per_atom': -0.965501308541666, 'is_stable': True}, {'formula_pretty': 'MoS2', 'material_id': 'mp-1434', 'formation_energy_per_atom': -0.965501308541666, 'is_stable': True}, {'formula_pretty': 'GaN', 'material_id': 'mp-804', 'formation_energy_per_atom': -0.65681686875, 'is_stable': True}, {'formula_pretty': 'GaN', 'material_id': 'mp-804', 'formation_energy_per_atom': -0.670049406875, 'is_stable': True}, {'formula_pretty': 'GaN', 'ma

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

Here are the formation energies (in eV per formula unit) of the materials in their most stable bulk form:

- Ag2O3: Data not found
- MgZn2: -0.13063 eV
- GaAs: -0.44628 eV
- GaN: -0.65682 eV
- SiC: -0.20388 eV
- BN: -1.46138 eV
- InSe: -0.74627 eV
- MoS2: -1.20156 eV
- BaTiO3: -3.37230 eV
- CsPbI3: -1.29613 eV

Please note that the formation energy for Ag2O3 was not found in the database.


In [5]:

model = "gpt-3.5-turbo-1106"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-3.5-turbo-1106
The formation energies of the following materials in their most stable bulk form are as follows:

Ag2O3: -2.074 eV/atom
MgZn2: -0.669 eV/atom
GaAs: -3.49 eV/atom
GaN: -3.43 eV/atom
SiC: -4.69 eV/atom
BN: -6.43 eV/atom
InSe: -0.291 eV/atom
MoS2: -1.89 eV/atom
BaTiO3: -3.13 eV/atom
CsPbI3: -1.90 eV/atom


In [6]:

model = "gpt-4"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    # streaming=True
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-4
The formation energy of a material is typically expressed in eV/atom or eV/formula unit. However, the value varies depending on the source, calculation method, and condition. Therefore, the exact values might not be universally agreed upon. Here are some approximate values:

1. Ag2O3: The formation energy for Ag2O3 is not commonly reported and may not be well-defined due to its instability.

2. MgZn2: -1.19 eV/atom

3. GaAs: -1.06 eV/atom 

4. GaN: -1.77 eV/atom 

5. SiC: -4.27 eV/atom 

6. BN: -7.37 eV/atom

7. InSe: -0.91 eV/atom 

8. MoS2: -2.01 eV/atom

9. BaTiO3: -7.57 eV/formula unit

10. CsPbI3: -3.43 eV/formula unit 

Please note that these values are approximate and the actual formation energies may differ slightly.


In [7]:

model = "gemini-pro"
llm = GoogleGenerativeAI(
    temperature=0.7,
    model=f"models/{model}",
    google_api_key=GOOGLE_API_KEY,
)


response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response)

gemini-pro
* Ag2O3: 0.48 eV/atom
* MgZn2: -0.05 eV/atom
* GaAs: -0.03 eV/atom
* GaN: -0.02 eV/atom
* SiC: -0.7 eV/atom
* BN: -0.5 eV/atom
* InSe: -0.02 eV/atom
* MoS2: -0.01 eV/atom
* BaTiO3: -0.02 eV/atom
* CsPbI3: -0.04 eV/atom


In [ ]:
model = "llama3"
llm = ChatOllama(
    model=model,
    temperature=0.7,
    # callbacks=[StreamingStdOutCallbackHandler()],
    num_predict=-2,
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)